In [ ]:
from typing import Tuple, Optional
from datetime import datetime, timedelta
import math
import scipy.constants as const

def get_quadrant(course):
    return course // 90 + 1

def degree_2_radian(angle):
    return (angle * math.pi) / 180

class Vessel:
    """Een klasse die een vaartuig beschrijft."""

    def __init__(self, name: str, position: Tuple[float, float], speed: float, true_course: float, length: float):
        self.name = name
        self.position = position #(lat, long)
        self.speed = speed
        self.true_course = true_course #degrees
        self.length = length

        self.rudder_position = 0 #radians
        self.rudder_timer_indicator = False
        self.rudder_timer = 0
        self.rate_of_turn = 0 #degrees/min
        
    def __str__(self):
        return (f"Name: {self.name}, position: {self.position}, "
                f"Speed: {self.speed} knots, Course: {self.true_course}°")
    
    def calculate_next_position(self, t1: datetime, t2: datetime):
        """Mid latitude methode"""

        time_difference = t2 - t1
        distance = (time_difference.total_seconds() / 3600) * self.speed

        departure = distance * math.sin(degree_2_radian(self.true_course)) #NM
        delta_latitude = distance * math.cos(degree_2_radian(self.true_course)) #NM

        delta_latitude = delta_latitude / 60 #degrees

        mid_latitude = self.position[0] + delta_latitude/2

        delta_longitude = departure / math.cos(degree_2_radian(mid_latitude)) #NM
        delta_longitude = delta_longitude / 60

        latitude = self.position[0] + delta_latitude
        longitude = self.position[1] + delta_longitude

        if longitude > 180.0:
            longitude = longitude - 360.0
        elif longitude < -180.0:
            longitude = longitude + 360

        if latitude > 90.0:    #Nakijken, dit klopt wss niet !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Koers zal wijzigen ipv graden
            latitude = latitude - 180.0
        elif latitude < -90.0:
            latitude = latitude + 180.0

        return (latitude, longitude)
    
    def change_position(self, latitude: Optional[float] = None, longitude: Optional[float] = None, position: Optional[Tuple[float, float]] = None):
        if position != None:
            self.position = position
        else:
            if latitude is not None and longitude is not None:
                self.position = (latitude, longitude)
            else:
                raise ValueError('Either lat and long or position!')
            
    def change_true_course(self, new_true_course):
        self.true_course = new_true_course

    def change_rudder_position(self, new_rudder_position):
        self.rudder_position = new_rudder_position

    def calculate_turning_circle(self):
        return self.length * ((self.speed * 0.51444)**2 / (const.g * math.tan(self.rudder_position)))

    def calculate_ROT(self):
        turning_circle = self.calculate_turning_circle()
        return (self.speed / ((turning_circle / 1852) * 3600)) # in rad/sec
    

In [29]:
MSC_Maya = Vessel(name='MSC Maya', position=(-40.41666667, 175.8333333333), speed=10, true_course=50, length=150)

In [30]:
new_position = MSC_Maya.calculate_next_position(datetime.now() - timedelta(hours=10), datetime.now())
print(new_position)

MSC_Maya.change_position(position=new_position)
print(MSC_Maya)


(-39.34535398701055, 177.49710428655493)
Name: MSC Maya, position: (-39.34535398701055, 177.49710428655493), Speed: 10 knots, Course: 50°


In [31]:
MSC_Maya.change_rudder_position(degree_2_radian(35))

In [32]:
print(f"pos: {MSC_Maya.rudder_position}, len:{MSC_Maya.length}, ")

pos: 0.6108652381980153, len:150, 


In [35]:
MSC_Maya.calculate_turning_circle() / 1852

0.3121564250273945

In [34]:
MSC_Maya.calculate_ROT() 

0.008898672444541237